In [1]:
import numpy as np
import pandas as pd
import scipy.io as sio
import sklearn
from sklearn import metrics, preprocessing
import math
from sklearn.neighbors import KNeighborsClassifier

data = sio.loadmat('Botswana.mat')['Botswana']
labels = sio.loadmat('Botswana_gt.mat')['Botswana_gt']
print(data.shape) #145, 145, 200
#print(labels.shape) #145, 145
#print(data[0][0])
print(len(labels[0]))

def extract_pixels(X, y):
  q = X.reshape(-1, X.shape[2])
  df = pd.DataFrame(data = q)
  df = pd.concat([df, pd.DataFrame(data = y.ravel())], axis=1)
  df.columns= [f'band{i}' for i in range(1, 1+X.shape[2])]+['class']
  df.to_csv('Dataset.csv')
  #print(X.shape)
  return df 
df = extract_pixels(data, labels)
df1 = np.transpose(df)
X1 = df.to_numpy()
X2 = np.transpose(X1)
#print(df)
#print(X1)
updatedX1 = []
for i in range(len(X1)):
    if(X1[i][145] != 0):
        updatedX1.append(X1[i])
print(len(updatedX1))
updatedX2 = np.transpose(updatedX1)
updatedX1=np.transpose(updatedX2)
X11 = updatedX1
X22 = updatedX2
print(len(X1))


FileNotFoundError: [Errno 2] No such file or directory: 'Botswana_gt.mat'

In [14]:
#df1.transpose().to_numpy().shape
band=X11
bandr=X22
bands=[]
bandnum = 15
print(bandr)

[[6181 6152 6087 ... 4026 3911 3948]
 [6266 6353 6192 ... 3937 3892 4010]
 [6142 6067 6049 ... 3486 3599 3633]
 ...
 [ 201  210  202 ...   35   32   44]
 [ 187  217  199 ...   41   42   38]
 [  14   14   14 ...    6    6    6]]


In [15]:
#KNN
from sklearn.svm import SVC
y=[]
y=bandr[145]
X_data=np.transpose(bandr)
svm =  SVC(C = 50, kernel = 'rbf', gamma = 'auto', cache_size = 10*1024)
svm.fit(X_data,y)



SVC(C=50, cache_size=10240, class_weight=None, coef0=0.0,
    decision_function_shape='ovr', degree=3, gamma='auto', kernel='rbf',
    max_iter=-1, probability=False, random_state=None, shrinking=True,
    tol=0.001, verbose=False)

In [16]:
#Competence and knn
kk=7
train,competence,test=np.split(X_data,[int(.01*len(X_data)),int(0.11*len(X_data))])
print(train)
train_y=np.transpose(train)
train_y=train_y[len(train_y)-1]
competence_y=np.transpose(competence)
competence_y=competence_y[len(competence_y)-1]
test_y=np.transpose(test)
test_y=test_y[len(test_y)-1]
train=np.transpose(train)
train=np.transpose(train[0:(len(train)-1)])
print(len(train_y))
competence=np.transpose(competence)
competence=np.transpose(competence[0:(len(competence)-1)])
test=np.transpose(test)

test=np.transpose(test[0:(len(test)-1)])
print(len(competence_y))
print(len(competence))
                  
                  
def competences(train,competence):
    regioncomp=[]
    m=[]
    for i in range(0,len(competence)):
        m.append(np.linalg.norm(train-competence[i]))
    bandind=[0]*(len(competence)+1)
    for i in range(0,len(competence)):
        bandind[i]=i
    
    for i in range(0,len(competence)):
        for j in range(0,len(competence)-i-1):
            if(m[j]>m[j+1]):
                temp=m[j]
                m[j]=m[j+1]
                m[j+1]=temp
                temp=bandind[j]
                bandind[j]=bandind[j+1]
                bandind[j+1]=temp
                temp=competence_y[j]
                competence_y[j]=competence_y[j+1]
                competence_y[j+1]=temp
                
                
    for i in range(0,kk):
        regioncomp.append(competence[bandind[i]])
    return regioncomp,competence_y

[[6181 6266 6142 ...  201  187   14]
 [6152 6353 6067 ...  210  217   14]
 [6087 6192 6049 ...  202  199   14]
 ...
 [7199 7114 6991 ...  218  219   14]
 [7410 7447 7495 ...  266  262   14]
 [8902 9300 9350 ...  293  301   14]]
32
325
325


In [17]:
from sklearn import metrics, preprocessing
from sklearn.metrics import accuracy_score
from sklearn.ensemble import RandomForestClassifier
from sklearn.datasets import make_classification

In [18]:
#Here X=X2.transpose
comb=[]
bands=[]
bandNumber = []
kk=7
y=bandr[145]
t = list(range(1, 146))
for i in range(0,bandnum):
    classifier1 = 0
    classifier2 = 0
    maxAcc1 = -1
    maxAcc2 = -1
    maxBand1 = -1
    maxBand2 = -1
    tempRegion = []
    print(bandNumber)
    for j in range(0,len(bandr)-1):
        comb=[]
        #comb=bands
        for z in bands:
            comb.append(z)
        comb.append(bandr[j])
        X=np.transpose(comb)
        
        #svm =  SVC(C = 50, kernel = 'rbf', gamma = 'auto', cache_size = 10*1024)
        #svm.fit(X,y)
        
        clf = RandomForestClassifier(n_estimators = 10, max_depth=2, random_state=0)
        clf.fit(X, y)
        
        classifier = KNeighborsClassifier(n_neighbors=kk)
        classifier.fit(X, y)
        #print("Trained", i, j)
        
        samnum=len(train)
        for k in range(0,samnum):
            #print(bandNumber)
            region,y_test=competences(train[k],competence)
            #print(region)
            regionDash = np.transpose(region)
            for l in range(len(bandNumber)):
                tempRegion.append(regionDash[bandNumber[l]-1])
            tempRegion.append(regionDash[j])
            region = tempRegion
            
            #y_pred1 = svm.predict(np.transpose(region))
            y_pred1 = clf.predict(np.transpose(region))
            y_pred2 = classifier.predict(np.transpose(region))
            
            accuracy1 = accuracy_score(y_test[:len(y_pred1)], y_pred1)
            accuracy2 = accuracy_score(y_test[:len(y_pred1)], y_pred2)

            if(accuracy1 > accuracy2):
                classifier1 += 1
                if(accuracy1 >= maxAcc1):
                    maxAcc1 = accuracy1
                    #maxBand1 = j+1
                    maxBand1 = t[j]
            else:
                classifier2 += 1
                if(accuracy2 >= maxAcc2):
                    maxAcc2 = accuracy2
                    #maxBand2 = j+1
                    maxBand2 = t[j]
                    
            region = []
            regionDash = []
            tempRegion = []
                
    if(classifier1 > classifier2):
        bandNumber.append(maxBand1)
        #bandr.remove(bandr[maxBand1 - 1])
        bandr = np.delete(bandr, maxBand1 - 1, axis = 0)
        bands.append(bandr[maxBand1 - 1])
        t.remove(maxBand1)
        #t.remove(t[maxBand1])
        #numpy.append(bands,bandr[maxBand1 - 1],axis = 0)
    else:
        bandNumber.append(maxBand2)
        #bandr.remove(bandr[maxBand2 - 1])
        bandr = np.delete(bandr, maxBand2 - 1, axis = 0)
        bands.append(bandr[maxBand2 - 1])
        t.remove(maxBand2)
        #t.remove(t[maxBand2])
        #numpy.append(bands,bandr[maxBand2 - 1],axis = 0)

print(bandNumber)

[]
[48]
[48, 60]
[48, 60, 80]
[48, 60, 80, 72]
[48, 60, 80, 72, 57]
[48, 60, 80, 72, 57, 68]
[48, 60, 80, 72, 57, 68, 107]
[48, 60, 80, 72, 57, 68, 107, 120]
[48, 60, 80, 72, 57, 68, 107, 120, 87]
[48, 60, 80, 72, 57, 68, 107, 120, 87, 131]
[48, 60, 80, 72, 57, 68, 107, 120, 87, 131, 122]
[48, 60, 80, 72, 57, 68, 107, 120, 87, 131, 122, 111]
[48, 60, 80, 72, 57, 68, 107, 120, 87, 131, 122, 111, 103]
[48, 60, 80, 72, 57, 68, 107, 120, 87, 131, 122, 111, 103, 116]
[48, 60, 80, 72, 57, 68, 107, 120, 87, 131, 122, 111, 103, 116, 12]


In [ ]:
print(len(bandr))
print(len(bands))
print(bandNumber)
#14:22

In [20]:
bands = bandNumber
#bands = [1, 2, 3, 4, 5, 6, 7, 8, 9]
xCheck = []
for i in bands:
    xCheck.append(X22[i-1])
xCheck.append(X22[145])
xDash = np.transpose(xCheck)
#print(len(xCheck[0]))
#print(xDash)
xx = []
for i in range(len(xDash)):
    if(xDash[i][9] != 0):
        xx.append(xDash[i])
#print(x)
X = np.array(xx)[:,:15]
y = np.array(xx)[:,15:]
from sklearn.model_selection import train_test_split
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.20)
names = [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14]
from sklearn.neighbors import KNeighborsClassifier
k = 9
classifier = KNeighborsClassifier(n_neighbors=k)
classifier.fit(X_train, y_train)
y_pred = classifier.predict(X_test)
#print(y_pred)
result = metrics.confusion_matrix(y_test, y_pred)
#print(result)
print(classification_report(y_test, y_pred))
print(accuracy_score(y_test, y_pred))

              precision    recall  f1-score   support

           1       1.00      1.00      1.00        60
           2       0.96      1.00      0.98        25
           3       0.86      0.98      0.91        49
           4       0.64      0.83      0.72        36
           5       0.81      0.80      0.80        64
           6       0.65      0.55      0.59        55
           7       1.00      1.00      1.00        51
           8       0.71      0.94      0.81        36
           9       0.78      0.71      0.74        59
          10       0.75      0.72      0.74        58
          11       0.94      0.91      0.93        56
          12       0.41      0.52      0.45        29
          13       0.60      0.47      0.53        55
          14       0.67      0.35      0.46        17

    accuracy                           0.79       650
   macro avg       0.77      0.77      0.76       650
weighted avg       0.79      0.79      0.78       650

0.7861538461538462


C:\Users\AAYUSH AGARWAL\Anaconda3\lib\site-packages\ipykernel_launcher.py:25: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
